In [7]:
!pip install git+https://github.com/BioMikeUkr/PyJAdES.git


  Cloning https://github.com/BioMikeUkr/PyJAdES.git to /tmp/pip-req-build-epsukiu2
  Running command git clone --filter=blob:none --quiet https://github.com/BioMikeUkr/PyJAdES.git /tmp/pip-req-build-epsukiu2
  Resolved https://github.com/BioMikeUkr/PyJAdES.git to commit f5ea6bae0c83e75591752e931449004e5f216d43
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
# Install OpenSSL (for Ubuntu/Debian systems)
#!apt install openssl

# Step 1: Create a directory to store certificates and keys
!mkdir certificates

# Step 2: Generate a private key for the Root CA (Root Certificate Authority)
!openssl genpkey -algorithm RSA -out certificates/rootCA_key.pem

# Step 3: Generate a self-signed Root CA certificate, valid for 1024 days
!openssl req -x509 -new -nodes -key certificates/rootCA_key.pem -sha256 -days 1024 -out certificates/rootCA_cert.pem \
  -subj "/C=LT/ST=Vilnius/L=Vilnius/O=MyRootCA/OU=RootCA/CN=My Root CA/emailAddress=rootca@example.com"

# Step 4: Generate a private key for the intermediate certificate
!openssl genpkey -algorithm RSA -out certificates/intermediateCA_key.pem

# Step 5: Create a Certificate Signing Request (CSR) for the intermediate certificate
!openssl req -new -key certificates/intermediateCA_key.pem -out certificates/intermediateCA.csr \
  -subj "/C=LT/ST=Vilnius/L=Vilnius/O=MyIntermediateCA/OU=IntermediateCA/CN=Intermediate CA/emailAddress=intermediateca@example.com"

# Step 6: Sign the intermediate certificate using the Root CA certificate (valid for 700 days)
!openssl x509 -req -in certificates/intermediateCA.csr -CA certificates/rootCA_cert.pem -CAkey certificates/rootCA_key.pem \
  -CAcreateserial -out certificates/intermediateCA_cert.pem -days 700 -sha256

# Step 7: Generate a private key for the user certificate
!openssl genpkey -algorithm RSA -out certificates/new_cert_key.pem

# Step 8: Create a Certificate Signing Request (CSR) for the user certificate
!openssl req -new -key certificates/new_cert_key.pem -out certificates/new_cert.csr \
  -subj "/C=LT/ST=Vilnius/L=Vilnius/O=BioMike/OU=BioMike/CN=Mykhailo Shtopko/emailAddress=emailAddress@gmail.com"

# Step 9: Sign the user certificate using the intermediate certificate (valid for 500 days)
!openssl x509 -req -in certificates/new_cert.csr -CA certificates/intermediateCA_cert.pem -CAkey certificates/intermediateCA_key.pem \
  -CAcreateserial -out certificates/new_cert_signed.pem -days 500 -sha256

# Step 10: Verify the signed user certificate using the Root CA and intermediate certificate
!openssl verify -CAfile certificates/rootCA_cert.pem -untrusted certificates/intermediateCA_cert.pem certificates/new_cert_signed.pem


mkdir: cannot create directory ‘certificates’: File exists
.+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.............+.....+...+.......+...........+.+.........+..+.......+..+.........+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*..+.+..+.+............+..............+.........+.+......+.....+..........+.....+.......+.....+...+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
.+.....+.........+....+..+.+..+...+.......+..+.........+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*...+.....+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*.....+......+......+...+..+.........+...............+....+...+.....+...+....+......+..+...+.......+...+.........+.....+..........+...+...+..+....+.......................+......+.+...+.....+...+.......+..+....+..+.........+.+.....+....+.........+.....+...............+....+......+...........+.+..+......+....+..+.++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [11]:
from pyjades import JAdESSigner, JAdESValidator

payload_to_sign = "Important data"
additional_signed_headers = {
    "sigPl": {
        "addressCountry": "LT",
        "addressLocality": "Vilnius",
        "addressRegion": "Vilnius",
        "postOfficeBoxNumber": "24",
        "postalCode": "LT-03225",
        "streetAddress": "Naugarduko g. 24"
    }
}

additional_etsiU = None
signer = JAdESSigner(
    key_file="certificates/new_cert_key.pem",
    cert_file="certificates/new_cert_signed.pem",
    intermediate_cert_file="certificates/intermediateCA_cert.pem"
)


signer.sign(payload_to_sign, additional_signed_headers, additional_etsiU, output_file="signed_dock.json")

# JAdESValidator
validator = JAdESValidator('signed_dock.json')
validator.validate(output_file="validation_result.json")

Signature saved in 'signed_dock.json'
Certificate .cache_dir/tsa.crt already exists in cache.
Certificate .cache_dir/cacert.pem already exists in cache.
Validation result saved in validation_result.json
